In [4]:
import tensorflow as tf

class DualAutoencoder(tf.keras.Model):
  def __init__(self, input_dim, latent_dim, noise_dim):
    super(DualAutoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      # Define your encoder architecture here (e.g., dense layers)
    ])
    self.decoder_clean = tf.keras.Sequential([
      # Define your decoder architecture for clean data (e.g., dense layers)
    ])
    self.decoder_noisy = tf.keras.Sequential([
      # Define your decoder architecture for noisy data (e.g., dense layers with noise injection)
    ])

  def call(self, inputs):
    latent = self.encoder(inputs)
    clean_recon = self.decoder_clean(latent)
    noisy_recon = self.decoder_noisy(latent)
    return clean_recon, noisy_recon

# ... (rest of the code for training and spectral clustering)


In [5]:
import tensorflow as tf

class DualAutoencoder(tf.keras.Model):
    def __init__(self, input_shape, latent_dim):
        super(DualAutoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(latent_dim, activation='relu')
        ])
        self.original_decoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=latent_dim),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(input_shape[0], activation='sigmoid')
        ])
        self.noisy_decoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=latent_dim),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(latent_dim, activation='relu'),  # Adding noise to latent representation
            tf.keras.layers.Dense(input_shape[0], activation='sigmoid')
        ])

    def call(self, inputs):
        latent_representation = self.encoder(inputs)
        reconstructed_original = self.original_decoder(latent_representation)
        reconstructed_noisy = self.noisy_decoder(latent_representation)
        return reconstructed_original, reconstructed_noisy

# Example usage:
input_shape = (784,)  # Example input shape for MNIST data
latent_dim = 64  # Example latent dimension
dual_autoencoder = DualAutoencoder(input_shape, latent_dim)
dual_autoencoder.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for reconstruction
dual_autoencoder.build((None, *input_shape))
dual_autoencoder.summary()



Model: "dual_autoencoder_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 64)                242112    
                                                                 
 sequential_4 (Sequential)   (None, 784)               242832    
                                                                 
 sequential_5 (Sequential)   (None, 784)               108752    
                                                                 
Total params: 593,696
Trainable params: 593,696
Non-trainable params: 0
_________________________________________________________________


In [6]:
import torch.nn.functional as F

class DualAutoencoderLoss(nn.Module):
    def __init__(self, reconstruction_weight=1.0, regularization_weight=0.001):
        super(DualAutoencoderLoss, self).__init__()
        self.reconstruction_weight = reconstruction_weight
        self.regularization_weight = regularization_weight

    def forward(self, x_original, x_noisy, x_decoded, x_encoded):
        # Reconstruction loss (MSE loss between original and decoded inputs)
        reconstruction_loss = F.mse_loss(x_decoded, x_original, reduction='mean')

        # Regularization loss (e.g., L2 regularization on the encoder weights)
        regularization_loss = 0.0
        for param in model.encoder.parameters():
            regularization_loss += torch.norm(param, p=2)

        # Total loss
        loss = self.reconstruction_weight * reconstruction_loss + self.regularization_weight * regularization_loss
        return loss

# Example usage:
# criterion = DualAutoencoderLoss()
# loss = criterion(x_original, x_noisy, x_decoded, x_encoded)
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()


ModuleNotFoundError: No module named 'torch'

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.csgraph import laplacian
from scipy.linalg import eigh

def spectral_clustering(data, num_clusters):
    # Step 1: Compute similarity graph (affinity matrix)
    similarity_matrix = pairwise_distances(data, metric='euclidean')
    affinity_matrix = np.exp(-similarity_matrix ** 2 / (2.0 * np.std(similarity_matrix) ** 2))
    
    # Step 2: Compute graph Laplacian
    laplacian_matrix = laplacian(affinity_matrix, normed=True)
    
    # Step 3: Compute eigenvalues and eigenvectors of Laplacian
    eigenvalues, eigenvectors = eigh(laplacian_matrix)
    
    # Step 4: Sort eigenvalues and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Step 5: Perform K-means clustering on selected eigenvectors
    kmeans = KMeans(n_clusters=num_clusters)
    cluster_labels = kmeans.fit_predict(sorted_eigenvectors[:, 1:num_clusters+1])  # Exclude the first eigenvector
    
    return cluster_labels

# Example usage:
# data: Your data matrix (n_samples x n_features)
# num_clusters: Number of clusters
# cluster_labels = spectral_clustering(data, num_clusters)
